### 对比不同feature区分有意识和无意识状态的能力

In [ ]:
# 抓取LOC和ROC
# 0428: 在event这一行中精确到s，虽然LOC精确到s，但是诱导给药的精度是m，因此LOC的实际精度还是m
# 0516: 在event这一行中精确到s，精度同上
# 0517: 在event这一行中精确到m
# 0518: 在event这一行中精确到m
# 0519: 在event这一行中精确到m
# 0523～0526: 在event表格中有专门记载

In [ ]:
from datetime import datetime

import mne

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

from bis_lib import *
from bis_lib_ML import feature_norm

In [ ]:
# 可以使用supervised learning进一步提升区分C和NC状态的能力
# feature: PE + LZC + (SVD)
# label: C/NC

In [ ]:
from scipy.fft import fft
from scipy.signal import blackman, hamming, detrend
def get_bispectrum(eeg_data):
    L = 1
    w = hamming(eeg_data.shape[-1])
    ywf = fft(w * detrend(eeg_data, type='constant'))
    print(ywf.shape)
    bispectrum = np.zeros((ywf.shape[0], ywf.shape[1], ywf.shape[2]//2, ywf.shape[2]//2))
    for i in range(bispectrum.shape[-2]):
        for j in range(bispectrum.shape[-1]):
            bispectrum[:,:,i,j] = np.abs(ywf[:,:,i] * ywf[:,:,j] * np.conj(ywf[:,:,i+j]))
    return bispectrum

In [ ]:
def get_trispectrum(eeg_data):
    L = 1
    w = hamming(eeg_data.shape[-1])
    ywf = fft(w * detrend(eeg_data, type='constant'))
    print('ywf.shape', ywf.shape)
    trispectrum = np.zeros((ywf.shape[0], ywf.shape[1], ywf.shape[2]//3, ywf.shape[2]//3, ywf.shape[2]//3))
    for i in range(trispectrum.shape[-3]):
        for j in range(trispectrum.shape[-2]):
            for k in range(trispectrum.shape[-1]):
                trispectrum[:,:,i,j,k] = np.abs(ywf[:,:,i] * ywf[:,:,j] * ywf[:,:,k] * 
                                                np.conj(ywf[:,:,i+j]) * np.conj(ywf[:,:,i+k]) * np.conj(ywf[:,:,j+k]) * ywf[:,:,i+j+k])
    return trispectrum
trispectrum = get_trispectrum(eeg_data)
print(trispectrum.shape)

In [ ]:
data_info = {}
draw_c_nc = {}
xy_learning = {}
feature_clean = True
fs = 1000
exp_dates = ['0428', '0516', '0517', '0518', '0519', '0523', '0524', '0525', '0526']

for exp_date in exp_dates:
# for exp_date in ['0526']:
    path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
    exp = get_exp(path_root, exp_date)
    eeg_start_time, eeg_end_time = get_eeg_time(path_root, exp)
    csv_path = '../csv_files/{}'.format(exp_date)
    for i in range(len(exp)):
    # for i in range(1):
        eeg = csv2eeg(path_root, exp[i])
        print('eeg.shape', eeg.shape)
        print('duration(s):', eeg.shape[1]/fs)
        data_info['exp{}_duration'.format(i)] = eeg.shape[1]/fs
        tmp = datetime.strptime(eeg_end_time[i], '%Y-%m-%d %H:%M:%S') - datetime.strptime(eeg_start_time[i], '%Y-%m-%d %H:%M:%S')
        data_info['exp{}_pack_loss'.format(i)] = tmp.seconds - eeg.shape[1]/fs

        ch_num = eeg.shape[0]
        # for mne object
        ch_names = ['EEG{}'.format(j) for j in range(ch_num)]
        ch_types = ['eeg' for i in range(ch_num)]
        info = mne.create_info(ch_names, ch_types=ch_types, sfreq=fs)
        print(info)

        data_info['exp{}_chnum'.format(i)] = eeg.shape[0]
        data_info['exp{}_filename'.format(i)] = exp[i]

        my_raw = mne.io.RawArray(eeg, info)
        my_raw.filter(l_freq=0.5, h_freq=30)
        # draw_eeg_psd(my_raw, ch_names, exp[i])

        if exp_date in ['0523', '0524', '0525', '0526']:
            xls_info = process_xls_new(csv_path, exp_id=i, eeg_start_time=eeg_start_time, eeg_end_time=eeg_end_time)
        else:
            xls_info = process_xls(csv_path, exp_id=i, eeg_start_time=eeg_start_time, eeg_end_time=eeg_end_time)
            
        eeg_data = eeg_segment(my_raw[:][0], fs, seg_length=60, eeg_start_time=eeg_start_time[i])

        eeg_feature_dict = {}
        if feature_clean:
            pe = np.load('../npy_files/feature_clean/PE_{}_{}.npy'.format(exp_date, exp[i]))
            # se = np.load('../npy_files/feature_clean/SE_{}_{}.npy'.format(exp_date, exp[i]))
            lzc = np.load('../npy_files/feature_clean/LZC_{}_{}.npy'.format(exp_date, exp[i]))
            svd = np.load('../npy_files/feature_clean/SVD_{}_{}.npy'.format(exp_date, exp[i]))
        else:
            pe = np.load('../npy_files/PE_{}_{}.npy'.format(exp_date, exp[i]))
            # se = np.load('../npy_files/SE_{}_{}.npy'.format(exp_date, exp[i]))
            lzc = np.load('../npy_files/LZC_{}_{}.npy'.format(exp_date, exp[i]))
            svd = np.load('../npy_files/SVD_{}_{}.npy'.format(exp_date, exp[i]))

        eeg_feature_dict['PE'] = np.mean(pe, axis=1)
        # eeg_feature_dict['SE'] = np.mean(pe, axis=1)
        eeg_feature_dict['LZC'] = np.mean(lzc, axis=1)
        eeg_feature_dict['SVD_0'] = svd[0,:]
        eeg_feature_dict['SVD_1'] = svd[1,:]
        eeg_feature_dict['SVD_2'] = svd[2,:]
        eeg_feature_dict['SVD_3'] = svd[3,:]

        # draw_results(exp_date, exp[i], xls_info, eeg_feature_dict)
        for key in eeg_feature_dict.keys():
            draw_c_nc['{}_{}_{}_c'.format(key, exp_date, exp[i])], draw_c_nc['{}_{}_{}_nc'.format(key, exp_date, exp[i])], xy_learning['{}_{}_{}_feature'.format(key, exp_date, exp[i])], xy_learning['{}_{}_{}_label'.format(key, exp_date, exp[i])] = draw_discrimination(eeg_feature_dict, xls_info, key, exp_date, exp[i])
        key = 'bis'
        draw_c_nc['{}_{}_{}_c'.format(key, exp_date, exp[i])], draw_c_nc['{}_{}_{}_nc'.format(key, exp_date, exp[i])], xy_learning['{}_{}_{}_feature'.format(key, exp_date, exp[i])], xy_learning['{}_{}_{}_label'.format(key, exp_date, exp[i])] = draw_discrimination(eeg_feature_dict, xls_info, key, exp_date, exp[i])

In [ ]:
# 根据不同的任务，有不同的被试数据需要舍弃
# 目前feature是在这个函数里面选择的
# 因此，方便起见，这个函数没有写进*_lib.py里
def get_model_data(exp_date_test, exp_idx_test):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    # key_bkp = ['PE', 'LZC', 'SVD_0', 'SVD_1', 'SVD_2', 'SVD_3']
    # key_bkp = ['bis']
    # key_bkp = ['PE']
    # key_bkp = ['LZC']
    # key_bkp = ['SVD_0']
    # key_bkp = ['SVD_1']
    # key_bkp = ['SVD_2']
    key_bkp = ['SVD_3']
    for exp_date in exp_dates:
        path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
        exp = get_exp(path_root, exp_date)
        eeg_start_time, eeg_end_time = get_eeg_time(path_root, exp)
        csv_path = '../csv_files/{}'.format(exp_date)
        for i in range(len(exp)):
            for j in range(len(xy_learning['{}_{}_{}_feature'.format(key_bkp[0], exp_date, exp[i])])):
                tmp = []
                for key in key_bkp:
                    tmp.append(xy_learning['{}_{}_{}_feature'.format(key, exp_date, exp[i])][j])
                if exp_date==exp_date_test and i==exp_idx_test:
                    test_x.append(tmp)
                    test_y.append(xy_learning['{}_{}_{}_label'.format(key_bkp[0], exp_date, exp[i])][j])      
                else:      
                    train_x.append(tmp)
                    train_y.append(xy_learning['{}_{}_{}_label'.format(key_bkp[0], exp_date, exp[i])][j])
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)
    print('train_x.shape', train_x.shape)
    print('test_x.shape', test_x.shape)
    print('train_y.shape', train_y.shape)
    print('test_y.shape', test_y.shape)
    return train_x, train_y, test_x, test_y

In [ ]:
acc_list = []
f1_list = []
for exp_date in exp_dates:
    path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
    exp = get_exp(path_root, exp_date)
    for i in range(len(exp)): 
        train_x, train_y, test_x, test_y = get_model_data(exp_date_test=exp_date, exp_idx_test=i)
        train_x, test_x = feature_norm(train_x, test_x)
        clf = SVC(kernel='rbf', max_iter=1000000, class_weight='balanced')
        clf.fit(train_x, train_y)
        acc_list.append(accuracy_score(test_y, clf.predict(test_x))) 
        f1_list.append(f1_score(test_y, clf.predict(test_x)))
print('mean(acc):{:.3f}'.format(np.mean(np.array(acc_list))))
print('mean(f1):{:.3f}'.format(np.mean(np.array(f1_list))))
print('std(acc):{:.3f}'.format(np.std(np.array(acc_list))))
print('std(f1):{:.3f}'.format(np.std(np.array(f1_list))))

In [ ]:
# 特征归一化，留一被试法，平均acc和f1
feature_str = ['BIS', 'MIX', 'PE', 'LZC', 'SVD_0', 'SVD_1', 'SVD_2', 'SVD_3']
# 降噪前
# acc_all = [0.789, 0.839, 0.806, 0.678, 0.773, 0.722, 0.754, 0.705]
# f1_all = [0.411, 0.667, 0.624, 0.388, 0.457, 0.477, 0.379, 0.409]
# 降噪后
acc_all = [0.789, 0.826, 0.807, 0.666, 0.762, 0.768, 0.702, 0.705]
f1_all = [0.411, 0.658, 0.630, 0.387, 0.402, 0.412, 0.291, 0.324]

In [ ]:
plt.figure(figsize=(8, 4))
color_bkp = ['forestgreen', 'darkorange',  'firebrick',
                'limegreen', 'royalblue', 'darkgrey', 'forestgreen', 'darkblue', 'purple']
x_bar = np.arange(len(acc_all))
width = 0.4
plt.bar(x_bar-width/2, acc_all, width=width, color=color_bkp[0], edgecolor='k', label='acc')
plt.bar(x_bar+width/2, f1_all, width=width, color=color_bkp[1], edgecolor='k', label='f1')
plt.bar(x_bar, np.zeros(len(acc_all)), tick_label=feature_str)

plt.xticks(fontproperties = 'Arial', size = 14)
plt.yticks(fontproperties = 'Arial', size = 14)
plt.ylabel('Metrics', font={'family':'Arial', 'size':16})
plt.xlabel('Feature', font={'family':'Arial', 'size':16})

plt.legend(loc=(1.05, 0.3), ncol=1, edgecolor='k', prop={'family':'Arial', 'size':14})
plt.tight_layout()
# plt.savefig('figs/loc_roc_class.png', dpi=300, bbox_inches='tight')
plt.show()